In [ ]:
# Processor requirements
# The first two cells must be code cells, inpath and outpath define the scan file location and output location
# inpath = '/dls/i16/data/2025/mm41580-1/processed/1114510_msmapper.nxs'
inpath = '/dls/i16/data/2025/mm41580-1/1114510.nxs'
inpath = '/dls/i16/data/2025/mm41697-1/1116348.nxs'
outpath = ""

# Plot msmapper HKL cuts

This notebook automatically loads remapped HKL volumes resulting from the MillerSpaceMapper software on I16 and plots cuts along the pricipal axes.

See https://confluence.diamond.ac.uk/display/I16/HKL+Mapping

This notebook is processed via the gda-zocalo-connector service and started at the end of a scan. The notebook is run in the `$ module load mmg` python environment giving it access to [mmg_toolbox](https://github.com/DiamondLightSource/mmg_toolbox) and other common python packages.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex

from mmg_toolbox import data_file_reader, module_info
from mmg_toolbox.utils import fitting
from mmg_toolbox.plotting.matplotlib import set_plot_defaults

set_plot_defaults()  # set custom matplotlib rcParams

def md(string):
    return display(Markdown(string))

print(module_info())

In [ ]:
# load scan file
scan = data_file_reader(inpath)

md(scan.format('# {beamline} Scan: {scan_number:.0f}\n*{filepath}*'))

# Time
start = scan('start_time')
stop = scan('end_time')
duration = stop - start

# Create table
tab = '| field | metadata |\n| --- | --- |\n'
for s in str(scan).split('\n'):
    if '=' in s:
        tab += '| %s |\n' % s.replace('=', '|')

tab += '|**start time** | %s |\n' % start.strftime('%Y-%m-%d %H:%M')
tab += '|**end time** | %s |\n' % stop.strftime('%Y-%m-%d %H:%M')
tab += '|**duration** | %s |\n' % duration

md(tab)

In [ ]:
md('# Default Plot')
if 'volume' not in scan.map:
    scan.plot()

In [ ]:
# If scan is a standard nexus file, not a remapped file, look for the remapped file in the processing folder

if 'volume' in scan.map:
    remap = scan
    print('File contains remapped data')
else:
    # Find remapped files
    proc_dir = os.path.dirname(scan.filename) + '/processed/'
    for ntries in range(10):
        # remapping may take some time, so keep checking until finished
        files = [proc_dir + file for file in os.listdir(proc_dir)]
        scn = str(scan.scan_number())
        mapper_files = [
            file for file in files
            if file.endswith('.nxs') and scn in file and
               'volume' in data_file_reader(file).map
        ]
        if len(mapper_files) > 0:
            remap = data_file_reader(mapper_files[0])
            break
        else:
            print('Remapped file does not exist yet, try again in 1 min')
            time.sleep(60)

print('Remapped file loaded: %s' % remap.filename)

In [ ]:
# Get reciprocal space data from file
h, k, l, vol = remap('h_axis, k_axis, l_axis, volume')
# metadata
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
hkl = scan.eval('array([h, k, l])')
vol_hkl = np.array([h.mean(), k.mean(), l.mean()])
pixel_size = remap('/entry0/instrument/pil3_100k/module/fast_pixel_direction')  # float, mm
detector_distance = remap('/entry0/instrument/pil3_100k/transformations/origin_offset')  # float, mm
# average angle subtended by each pixel
solid_angle = pixel_size ** 2 / detector_distance ** 2  # sr
vol = vol * solid_angle

md(remap.format("""
## metadata
 - hkl: ({h:.3f}, {k:.3f}, {l:.3f})
 - volume hkl: ({mean(h_axis):.3f}, {mean(k_axis):.3f}, {mean(l_axis):.3f})
 - pixel_size: {s_fast_pixel_direction}
 - detector_distance: {s_origin_offset}
## Reciprocal Space Volume
- h_axis: {s_h_axis}
- k_axis: {s_k_axis}
- l_axis: {s_l_axis}
- volume: {s_volume}
"""))

md(f'Each pixel is normalised by the solid angle: {solid_angle: .4g} sr\n\nThe max intensity is {vol.max():.4g} counts')


In [ ]:
# Plot histogram
cmap = plt.get_cmap()
cut_ratios=(1e-3, 1e-2, 1e-1)  # lines at points relative to max intensity
alphas = np.linspace(0.1, 1, len(cut_ratios))
max_volume = vol.max()

ax = plt.figure().add_subplot()
n, bins, patches = ax.hist(np.log10(vol[vol > 0].flatten()), 100)

# for cut, alpha in zip(cut_ratios, alphas):
#     logval = np.log10(cut * max_volume)
#     ax.axvline(logval, marker='', c='k')
#     for patch in patches:
#         if patch.xy[0] >= logval:
#             patch.set_color(cmap(alpha))

ax.set_xlabel('Log$_{10}$ Voxel Intensity')
ax.set_ylabel('N')
ax.set_title(title)

mode_background = 10 ** bins[np.argmax(n)]
md(f"most common intensity == mode ~= background = {mode_background:.4g}")

In [ ]:
# Volume statistics
nonzero_vol = vol[vol > 0]
background = nonzero_vol.min() + 1
signal = nonzero_vol.max() - background
ii, ij, ik = fitting.max_index(vol)
is_peak = (
    vol[vol > mode_background].size / vol.size > 0.01 and  # enough signal pixels to call a peak
    signal > 3 * np.sqrt(background)
)

md(f"""
- signal: {signal:.4g}
- background: {background:.4g}
- signal / sqrt(background): {signal / np.sqrt(background):.4g}
- signal i,j,k = {int(ii), int(ij), int(ik)}
- signal voxels = {vol[vol > mode_background].size} [{vol[vol > mode_background].size / vol.size:.3%}]
- is_peak: {is_peak}
""")

In [ ]:
if is_peak:
    L, K = np.meshgrid(l, k)
    plt.pcolormesh(K, L, vol[ii, :, :])
    plt.plot([k[ij]], l[ik], 'k+', markersize=16)
    plt.xlabel('k-axis (r.l.u.)')
    plt.ylabel('l-axis (r.l.u.)')
    plt.title(f'Max Intensity\nh = {h[ii]}, k = {k[ij]}, l = {l[ik]}')
    #plt.axis('image')
    plt.colorbar(label='Intensity')
else:
    md('*No Peak found*')

In [ ]:
# Create a window around the peak
if is_peak:
    window_size = 10  # voxels
    ws = (
        slice(ii-window_size, ii+window_size),
        slice(ij-window_size, ij+window_size),
        slice(ik-window_size, ik+window_size),
    )
else:
    ws = (slice(None), slice(None), slice(None))

In [ ]:
# Plot summed cuts
plt.figure(figsize=(18, 8), dpi=60)
plt.suptitle(title)

plt.subplot(131)
plt.plot(h, vol[:, ws[1], ws[2]].sum(axis=1).sum(axis=1))
plt.xlabel('h-axis (r.l.u.)')
plt.ylabel('sum axes [1,2]')
if is_peak:
    plt.title(f'k = {k[ij]}, l = {l[ik]}')

plt.subplot(132)
plt.plot(k, vol[ws[0], :, ws[2]].sum(axis=0).sum(axis=1))
plt.xlabel('k-axis (r.l.u.)')
plt.ylabel('sum axes [0,2]')
if is_peak:
    plt.title(f'h = {h[ii]}, l = {l[ik]}')

plt.subplot(133)
plt.plot(l, vol[ws[0], ws[1], :].sum(axis=0).sum(axis=0))
plt.xlabel('l-axis (r.l.u.)')
plt.ylabel('sum axes [0,1]')
if is_peak:
    plt.title(f'h = {h[ii]}, k = {k[ij]}')

In [ ]:
# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
plt.suptitle(title)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
K, H = np.meshgrid(k, h)
plt.pcolormesh(H, K, vol[:, :, ws[2]].sum(axis=2), shading='gouraud')
plt.xlabel('h-axis (r.l.u.)')
plt.ylabel('k-axis (r.l.u.)')
if is_peak:
    plt.title(f'l = {l[ik]}')
#plt.axis('image')
#plt.colorbar()

plt.subplot(132)
L, H = np.meshgrid(l, h)
plt.pcolormesh(H, L, vol[:, ws[1], :].sum(axis=1), shading='gouraud')
plt.xlabel('h-axis (r.l.u.)')
plt.ylabel('l-axis (r.l.u.)')
if is_peak:
    plt.title(f'k = {k[ij]}')
#plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(l, k)
plt.pcolormesh(K, L, vol[ws[0], :, :].sum(axis=0), shading='gouraud')
plt.xlabel('k-axis (r.l.u.)')
plt.ylabel('l-axis (r.l.u.)')
if is_peak:
    plt.title(f'h = {h[ii]}')
#plt.axis('image')
plt.colorbar(label='Intensity (a.u.)')
plt.show()

## Peak Fitting

In [ ]:
if is_peak:
    from mmg_toolbox.utils import fitting

    # slices through the volume summed accross several pixels
    h_slice = vol[:, ws[1], ws[2]].sum(axis=1).sum(axis=1) - mode_background
    k_slice = vol[ws[0], :, ws[2]].sum(axis=0).sum(axis=1) - mode_background
    l_slice = vol[ws[0], ws[1], :].sum(axis=0).sum(axis=0) - mode_background

    # mask background
    h_fit, h_slice_bk = h[h_slice > 0], h_slice[h_slice > 0]
    k_fit, k_slice_bk = k[k_slice > 0], k_slice[k_slice > 0]
    l_fit, l_slice_bk = l[l_slice > 0], l_slice[l_slice > 0]

    h_result = fitting.multipeakfit(h_fit, h_slice_bk, plot_result=True)
    k_result = fitting.multipeakfit(k_fit, k_slice_bk, plot_result=True)
    l_result = fitting.multipeakfit(l_fit, l_slice_bk, plot_result=True)

    print(h_result)
    print(k_result)
    print(l_result)
else:
    md('*No Peak Found*')

## Convert to cartesian coordinates (Q)


In [ ]:
# Fail if hkl values are not consistent
assert np.linalg.norm(vol_hkl - hkl) < 0.5, f"Measured hkl={hkl} is different from remapped hkl={vol_hkl}"

In [ ]:
from mmg_toolbox.diffraction.lattice import bmatrix, cal2theta

# Convert to Q
a, b, c, alpha, beta, gamma = scan('unit_cell')
energy = scan('incident_energy')

# Build reciprocal lattice in orthogonal basis
astar, bstar, cstar = bmatrix(a, b, c, alpha, beta, gamma)
kk, hh, ll = np.meshgrid(k, h, l)
q = astar * hh.reshape(-1, 1) + bstar * kk.reshape(-1, 1) + cstar * ll.reshape(-1, 1)
qx = q[:, 0].reshape(hh.shape)
qy = q[:, 1].reshape(hh.shape)
qz = q[:, 2].reshape(hh.shape)
# magnitude of wavevector-transfer Q=kf-ki
qmag = np.sqrt(qx **2 + qy ** 2 + qz **2)

In [ ]:
# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
plt.suptitle(title)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
plt.pcolormesh(qx.mean(axis=2), qy.mean(axis=2), vol[:, :, ws[2]].sum(axis=2), shading='gouraud')
plt.xlabel('Qx [A$^{-1}$]')
plt.ylabel('Qy [A$^{-1}$]')
plt.axis('image')
#plt.colorbar()

plt.subplot(132)
plt.pcolormesh(qx.mean(axis=1), qz.mean(axis=1), vol[:, ws[1], :].sum(axis=1), shading='gouraud')
plt.xlabel('Qx [A$^{-1}$]')
plt.ylabel('Qz [A$^{-1}$]')
plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(l, k)
plt.pcolormesh(qy.mean(axis=0), qz.mean(axis=0), vol[ws[0], :, :].sum(axis=0), shading='gouraud')
plt.xlabel('Qy [A$^{-1}$]')
plt.ylabel('Qz [A$^{-1}$]')
plt.axis('image')
plt.colorbar()

### |Q| and Two-Theta plots

In [ ]:
qmag2 = qmag.reshape(-1)
qvol = vol.reshape(-1)
bin_cen = np.arange(qmag2.min(), qmag2.max(), 0.001)
bin_edge = bin_cen + 0.005
bin_pos = np.digitize(qmag2, bin_edge) -1
bin_sum = [np.mean(qvol[bin_pos==n]) for n in range(len(bin_cen))]
tth = cal2theta(bin_cen, energy)

title2 = scan.format('#{scan_number:.0f}\n{(cmd|user_command|scan_command)}')

plt.figure()
plt.plot(bin_cen, bin_sum)
plt.title(title2)
plt.xlabel('|Q| A$^{-1}$')
plt.ylabel('Intensity')

plt.figure()
plt.plot(tth, bin_sum)
plt.title(title2)
plt.xlabel('Two-Theta [Deg]')
plt.ylabel('Intensity')

## Plot Geometry

In [ ]:
instrument = scan.instrument_model()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
instrument.plot_wavevectors(ax)
